# Trabalho Final
Neste trabalho, vamos buscar o reconhecimento dos dígitos da Linguagem de Sinais. Para tal, vamos nos utilizar de Modelos conhecidos de Deep Learning e também nos aventurar na criação de próprios.

------

## Imports

In [2]:
import os
import numpy as np
from random import sample, seed
seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,15) # Make the figures a bit bigger

# Keras imports
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

import inf619utils

Using TensorFlow backend.


# Dataset
O dataset é composto por 10 classes (dígitos de 0 a 9) com aproximadamente 205 imagens por classe. 
O conjunto foi dividido em 60% para treinamento, 15% para validação e 20% para teste.
As imagens estão divididas em blocos por classe.

** IMPORTANTE NÃO ALTERAR O NOME/LOCAL DAS IMAGENS** 

In [3]:
datasetDir = "./Dataset"
nbClasses = 10
input_shape = (100,100,3)

train_files = {}
val_files = {}
test_files = {}

train_files, val_files, test_files = inf619utils.splitData(datasetDir, nbClasses)

In [4]:
#plot the images from imgList
def plotImagesFromBatch(imgList):
    for i in range(len(imgList)):
        plotImage(imgList[i])


# Se quiser visualizar algum bloco de imagens, descomentar as linhas abaixo
# inf619utils.plotImages(val_files)
# inf619utils.plotImages(train_files)
# inf619utils.plotImages(test_files)

In [5]:
trainSetSize = inf619utils.getDatasetSize(train_files)
valSetSize = inf619utils.getDatasetSize(val_files)
testSetSize = inf619utils.getDatasetSize(test_files)

print("# images in Train set: ", trainSetSize)
print("# images in Val set: ", valSetSize)
print("# images in Test set: ", testSetSize)

# images in Train set:  1242
# images in Val set:  309
# images in Test set:  511


## Definição do modelo 

In [6]:
# Fire Module Definition
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

#SqueezeNet model definition
def SqueezeNet(input_shape):
    img_input = Input(shape=input_shape) #placeholder
    
    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='drop9')(x)

    x = Convolution2D(1000, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='loss')(x)

    model = Model(img_input, x, name='squeezenet')

    # Download and load ImageNet weights
    model.load_weights('./squeezenet_weights_tf_dim_ordering_tf_kernels.h5')
    
    return model    

In [7]:
for batch, labels in inf619utils.loadDatasetInBatches(train_files, batch_size=32, input_shape=input_shape, nbClasses=nbClasses):
    print(batch.shape, labels.shape)
    #plotImagesFromBatch(batch)
    break

(32, 100, 100, 3) (32, 10)



# Modificação do modelo 

In [8]:
# Definir o modelo base da squeezeNet 
squeezeNetModel = SqueezeNet((100,100,3))

# Escolher a camada que será o ponto de partida 
x = squeezeNetModel.get_layer(name="fire9/concat").output

#print([layer.name for layer in squeezeNetModel.layers])
#print("\n\nFreeze layers up until ", squeezeNetModel.layers[-20].name)

for layer in squeezeNetModel.layers:
    layer.trainable = True#        layer.trainable = False

x = Convolution2D(1024, (1, 1), padding='valid', name='conv10_new')(x)
x = Activation('relu', name='relu_conv10_new')(x)
x = Dropout(0.5)(x)
x = Convolution2D(nbClasses, (1, 1), padding='valid', name='conv11_new')(x)
x = Activation('relu', name='relu_conv11_new')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='loss_new')(x)


# Não se esqueça de definir o nome modelo, onde baseSqueezeNetModel 
# é o modelo base da Squeeze que vc definiu ali em cima
model = Model(squeezeNetModel.inputs, x, name='squeezenet_new')

# Treinamento do Modelo

In [9]:
from keras.optimizers import SGD, Adam
#Compile o modelo
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True), metrics=['accuracy'])

import keras.callbacks as callbacks

tbCallBack = callbacks.TensorBoard(log_dir = "./logs_squeeze8")
tbEarly = callbacks.EarlyStopping(monitor='val_acc',min_delta=0,patience=10,verbose=0, mode='auto')

In [10]:
#Definir tamanho do batch e número de épocas
batch_size = 32
epochs = 30

#Criação dos generators
trainGenerator = inf619utils.loadDatasetInBatches(train_files, batch_size = batch_size, input_shape=input_shape, nbClasses=nbClasses)
valGenerator = inf619utils.loadDatasetInBatches(val_files, batch_size = batch_size)

#Fit nos dados
hist = model.fit_generator(trainGenerator, 
                    steps_per_epoch= int(trainSetSize / batch_size), 
                    epochs = epochs,
                    validation_data = valGenerator,  
                    validation_steps = int(valSetSize / batch_size),
                    callbacks=[tbCallBack, tbEarly])

Epoch 1/30
38/38 [==============================] - 16s 420ms/step - loss: 2.4067 - acc: 0.1184 - val_loss: 2.2337 - val_acc: 0.2674
Epoch 2/30
38/38 [==============================] - 12s 306ms/step - loss: 2.2287 - acc: 0.2309 - val_loss: 2.1213 - val_acc: 0.3430
Epoch 3/30
38/38 [==============================] - 11s 289ms/step - loss: 2.0932 - acc: 0.2832 - val_loss: 1.9419 - val_acc: 0.4477
Epoch 4/30
38/38 [==============================] - 11s 289ms/step - loss: 1.8773 - acc: 0.3979 - val_loss: 1.6844 - val_acc: 0.4946
Epoch 5/30
38/38 [==============================] - 11s 302ms/step - loss: 1.5637 - acc: 0.4891 - val_loss: 1.2680 - val_acc: 0.6462
Epoch 6/30
38/38 [==============================] - 12s 328ms/step - loss: 1.2160 - acc: 0.6153 - val_loss: 0.8881 - val_acc: 0.8014
Epoch 7/30
38/38 [==============================] - 13s 341ms/step - loss: 0.8983 - acc: 0.7311 - val_loss: 0.6326 - val_acc: 0.8123
Epoch 8/30
38/38 [==============================] - 12s 316ms/step - 

# Aplicação do Modelo no Conjunto de Testes

In [12]:
#Criação do generator p/ o conjunto de teste
testGenerator = inf619utils.loadDatasetInBatches(test_files, batch_size=batch_size)

#Teste
metrics = model.evaluate_generator(testGenerator, 
                                   steps=int(testSetSize/batch_size), 
                                   verbose=1)

print("Test Loss ---> ", metrics[0])
print("Test Accuracy ---> ", metrics[1])    #Test is balanced, so Acc is normalized

15/15 [==============================] - 2s 114ms/step
Test Loss --->  0.09766870339711507
Test Accuracy --->  0.9666666666666667
